In [43]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
#import networkx as nx
import copy


import tensorflow as tf

from sklearn.model_selection import train_test_split
import numpy as np

import tensorflow.keras as k
from sklearn.utils import shuffle
import tensorflow.keras.backend as K

from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import linear_model
from collections import defaultdict
import random


from keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler, PowerTransformer

import Utilities as util


In [2]:
#Import Data

return_data = pd.read_csv('Data/return_data')

Firm_CR = pd.read_csv('Data/Firm_CR.csv')
Firm_CR.set_index(['yq', 'firmID'], inplace =True)
EPS_S = pd.read_csv('Data/EPS_S.csv')
EPS_S.set_index(['yq', 'firmID'], inplace =True)

Y_All= pd.read_csv('Data/Y_All.csv')
Y_All.set_index(['yq', 'firmID'], inplace =True)

no_firm = len(EPS_S.groupby('firmID').size())
no_quarter = len(EPS_S.groupby('yq').size())
no_analyst = EPS_S.shape[1]
print("no_firm:", no_firm, '\n'
      "no_quarter:", no_quarter,'\n' 
      "no_analyst:", no_analyst)

no_firm: 293 
no_quarter: 32 
no_analyst: 488


In [3]:
#Create Correlation Matrix and to be used for return similarity regularization
return_cor = return_data[['date', 'firmID', 'ret']]
return_cor = return_cor.set_index(['date', 'firmID']).unstack()
return_cor.columns = return_cor.columns.droplevel()

D_firm = util.Get_D(return_cor)

In [72]:
#Set hyper-parameters

lr = 1e-4
rank = 10
nc = rank
epochs = 5
batch_size = 128


seed = 3
verbose = 1

In [73]:
lamda_1 = 0.01
lamda_2 = 0.01
lamda  = 0.01

In [ ]:
tf.keras.backend.clear_session()


All_Train_predicted= {}
All_Test_predicted = {}


#for i in range(201602,201603):
#for i in (201501, 201502, 201503, 201504, 201601, 201602, 201603, 201604):
for i in (201603, 201604): 
    print(i)
    
    F_C = Firm_CR.iloc[Firm_CR.index.get_level_values('yq') <= i]
    N_Q = len(F_C.groupby('yq').size())
    
    scaler = StandardScaler()
    #F_power = pt.fit_transform(F_C)
    F_power = scaler.fit_transform(F_C.values)
    #F_power = pd.DataFrame(F_power, index=F_C.index, columns=F_C.columns)
    M_data = pd.DataFrame(F_power, index=F_C.index, columns=F_C.columns)
    
   
        
    Y_train = Y_All.iloc[Y_All.index.get_level_values('yq') < i].values.flatten()
    Y_test = Y_All.iloc[Y_All.index.get_level_values('yq') == i].values.flatten()
    
    W_original = util.create_W_matrix(M_data)
    
    X_tensor = M_data.values.reshape(N_Q, -1, M_data.shape[1])
    
    
    
    # creating data file and indices
    shape = np.array(X_tensor.shape)

    #getting true values and indices
    tr_vals = X_tensor[~np.isnan(X_tensor)]
    tr_idxs = np.argwhere(np.isnan(X_tensor) ==0)
    
       
    np.random.seed(seed)


    #set_session(device_count={"GPU": 0}, seed=seed)
    optim = k.optimizers.Adam(lr=lr)

    model = util.create_NLTC(shape, rank, nc)
    #model.compile(optim, loss=["mse"], metrics=["mae", mape_keras])
    model.compile(optim, loss=util.regularized_loss_1(model.weights[1], D_firm, lamda), metrics=["mse", "mae", util.mape_keras])
    #model.compile(optim, loss=util.regularized_loss(model.weights[0], model.weights[1], D_firm, lamda_1, lamda_2), metrics=["mse", "mae", util.mape_keras]) 
    
    hists = model.fit(
    x=util.transform(tr_idxs),
    y=tr_vals,
    verbose=verbose,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.1,
    callbacks=[k.callbacks.EarlyStopping(
        #monitor="val_mean_absolute_error", 
        monitor="val_mae",
        patience=20, 
        restore_best_weights=True)], );
    
    
    all_indices = np.argwhere(np.random.rand(X_tensor.shape[0], X_tensor.shape[1], X_tensor.shape[2]))
    all_imputed = model.predict(transform(all_indices), batch_size= 64, verbose=1)

    imp_mat_costco = all_imputed.reshape(M_data.shape[0], M_data.shape[1])
    
    X_imputed = (imp_mat_costco * (1-W_original)) + np.nan_to_num(M_data.values)
    X_imputed_df = pd.DataFrame(X_imputed, index =M_data.index, columns = M_data.columns )

    Imputed_matrices[i] = X_imputed_df
    
    X_imputed_df = X_imputed_df[list(data_all.columns)]
    
    X_train  =   X_imputed_df.iloc[X_imputed_df.index.get_level_values('yq') < i].values
    X_test=   X_imputed_df.iloc[X_imputed_df.index.get_level_values('yq') == i].values
    
    
    
    
    
    print('********* Model SVR ***********')
    clf = SVR(gamma='scale',  C=10, epsilon=0.00001)
    clf.fit(X_train, Y_train) 
    y_tr_pred = clf.predict(X_train)
    y_ts_pred = clf.predict(X_test)
    
    
    SVRtrain_result[i] = util.Get_performance(Y_train, y_tr_pred)
    SVRtest_result[i] = util.Get_performance(Y_test, y_ts_pred)
    
    

    print('********* Model XGBOOST ***********')
    model_xgb = XGBRegressor(silent=False, 
                      learning_rate=0.1,  
                      n_estimators=1000, 
                      reg_alpha = 0.001,
                      max_depth=5, 
                      gamma=.1)
    model_xgb.fit(X_train, Y_train)
    y_tr_pred = model_xgb.predict(X_train)
    y_ts_pred = model_xgb.predict(X_test)
    
    
    All_Train_predicted[i] = y_tr_pred
    All_Test_predicted[i] = y_ts_pred 
    
    XGBtrain_result[i] = util.Get_performance(Y_train, y_tr_pred)
    XGBtest_result[i] = util.Get_performance(Y_test, y_ts_pred)
    



  
    
   
    
    
   
    
    
    

201603


/home/ajim/miniconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/ajim/miniconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:990: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/home/ajim/miniconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:1020: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count


Epoch 1/5


/home/ajim/miniconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


5514/5514 [==============================] - 49s 9ms/step - loss: 0.9972 - mse: 0.9972 - mae: 0.6385 - mape_keras: 92.6642 - val_loss: 0.9806 - val_mse: 0.9806 - val_mae: 0.6309 - val_mape_keras: 95.4324
Epoch 2/5
5514/5514 [==============================] - 48s 9ms/step - loss: 0.9613 - mse: 0.9613 - mae: 0.6199 - mape_keras: 94.1729 - val_loss: 0.9366 - val_mse: 0.9366 - val_mae: 0.6153 - val_mape_keras: 98.1749
Epoch 3/5
 862/5514 [===>..........................] - ETA: 38s - loss: 0.9072 - mse: 0.9072 - mae: 0.6074 - mape_keras: 95.2436